In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
data = [('James','Smith','M',3000),
  ('Anna','Rose','F',4100),
  ('Robert','Williams','M',6200), 
]

columns = ["firstname","lastname","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()

#Example 1 mapPartitions()
def reformat(partitionData):
    for row in partitionData:
        yield [row.firstname+","+row.lastname,row.salary*10/100]
df.rdd.mapPartitions(reformat).toDF().show()

#Example 2 mapPartitions()
def reformat2(partitionData):
  updatedData = []
  for row in partitionData:
    name=row.firstname+","+row.lastname
    bonus=row.salary*10/100
    updatedData.append([name,bonus])
  return iter(updatedData)

df2=df.rdd.mapPartitions(reformat2).toDF("name","bonus")
df2.show()

+---------+--------+------+------+
|firstname|lastname|gender|salary|
+---------+--------+------+------+
|    James|   Smith|     M|  3000|
|     Anna|    Rose|     F|  4100|
|   Robert|Williams|     M|  6200|
+---------+--------+------+------+

+---------------+-----+
|             _1|   _2|
+---------------+-----+
|    James,Smith|300.0|
|      Anna,Rose|410.0|
|Robert,Williams|620.0|
+---------------+-----+



---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-303549684801372>:27
     24     updatedData.append([name,bonus])
     25   return iter(updatedData)
---> 27 df2=df.rdd.mapPartitions(reformat2).toDF("name","bonus")
     28 df2.show()

File /databricks/spark/python/pyspark/sql/session.py:113, in _monkey_patch_RDD.<locals>.toDF(self, schema, sampleRatio)
     78 @no_type_check
     79 def toDF(self, schema=None, sampleRatio=None):
     80     """
     81     Converts current :class:`RDD` into a :class:`DataFrame`
     82 
   (...)
    111     +---+
    112     """
--> 113     return sparkSession.createDataFrame(self, schema, sampleRatio)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(